
### Statistical modeling

This notebook is used to statistically model differences between physiology values and observe the effects and contribution of features.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import statsmodels.formula.api as smf
from IPython.display import clear_output
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import ipywidgets as widgets
from zenml import pipeline

from configs import configs
from configs.parser import ConfigParser
from data_manager.loaders import StructuredData
from steps import data_formatter, data_loader

In [ ]:
# Define physiological parameter
options = ["E", "ETR", "gsw", "PhiPS2", "SPAD"]
dropdown_configs = widgets.Dropdown(
    options=options,
    value=options[0],
    description='Select:',
    disabled=False,
)

# Display the widget
display(dropdown_configs)

In [ ]:
# Define dates to use
options = [1, 2, 3]
dropdown_dates = widgets.SelectMultiple(
    options=options,
    value=options,
    description='Select:',
    disabled=False,
)

# Display the widget
display(dropdown_dates)

Select values above and run cells bellow by using 
->  `Execute cell and below`

In [ ]:
data_loader = data_loader.with_options(enable_cache=True)
data_formatter = data_formatter.with_options(enable_cache=True)


@pipeline(enable_cache=True)  # type: ignore
def load_data() -> StructuredData:
	cfg_parser = ConfigParser()
	data = data_loader(cfg_parser.general().without_varieties(), cfg_parser.multispectral())
	data = data_formatter(data, cfg_parser.general(), cfg_parser.formatter())
	return data


def load_data_last_run(config_name: str):
	# Set the TOML config file as an environment variable (parsed in the pipelines)
	os.environ[configs.TOML_ENV_NAME] = str(configs.TOML_DIR / f"reg/{config_name}.toml")
	# Run the pipeline only the first time to load the data
	load_data()
	clear_output()

	last_run = load_data.model.last_successful_run
	data = last_run.steps["data_formatter"]
	data = data.outputs["data"].load()
	return data

In [ ]:
config_name = dropdown_configs.value
data = load_data_last_run(config_name)

In [ ]:
target = data.target.value.to_frame(data.target.name).reset_index(drop=True)
target[target<0] = 0.00001
meta = data.meta.reset_index(drop=True)
df = pd.concat([meta, target], axis=1)
print(df.head())

# Normalize data (select one and test)
df[data.target.name], _ = stats.boxcox(df[data.target.name]) # Boxcox
# df[data.target.name] = np.log(df[data.target.name]) # Log
# df[data.target.name] = np.sqrt(df[data.target.name]) # sqrt
# df[data.target.name] = np.cbrt(df[data.target.name]) # cbrt
# df[data.target.name] = 1 / df[data.target.name]  # inv


In [ ]:
# Plot a histogram
plt.hist(df[data.target.name], bins='auto')
plt.title(f'Histogram of {data.target.name}')
plt.show()

# Plot a Q-Q plot
stats.probplot(df[data.target.name], plot=plt)
plt.title(f"Q-Q plot of {data.target.name}")
plt.show()

In [ ]:
# Convert data to appropriate format
date_map = {"2022_06_15": 1, '2022_07_11': 2, '2022_07_20': 3}
# Convert 'dates' to ordinal form
df['dates'] = df['dates'].apply(lambda x: date_map[x])
# Define the groups for the random effects
df["blocks"] = df["blocks"].astype(str)
# Optionally remove particular date:
dates_to_keep = dropdown_dates.value
df = df[df["dates"].isin(dates_to_keep)]

# Define the formula for the model
formula = f'{data.target.name} ~ treatments + varieties'

# Create the mixed linear model
model = smf.mixedlm(formula, df, groups=df['blocks'], re_formula="~dates")

# Fit the model
result = model.fit(reml=True)

# Print the summary
print(result.summary())

# Plot the residuals
plt.scatter(result.fittedvalues, result.resid)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

In [ ]:
# Create a new grouping variable that combines treatments and varieties
df['treatment_variety'] = df['treatments'].astype(str) + "_" + df['varieties'].astype(str)

# Perform pairwise comparisons on the residuals
posthoc = pairwise_tukeyhsd(endog=result.resid, groups=df['treatment_variety'], alpha=0.05)

# Create a DataFrame from the Tukey HSD results
df_result = pd.DataFrame(data=posthoc._results_table.data[1:], columns=posthoc._results_table.data[0])

# Create a pivot table of p-values
pivot_table = df_result.pivot(index='group1', columns='group2', values='p-adj')

# Convert p-values to numeric and round to three decimal places
pivot_table = pivot_table.apply(pd.to_numeric, errors='coerce').round(3)

# Create a heatmap
sns.heatmap(pivot_table, annot=True, center=0)
plt.show()

In [ ]:

unique_dates = df['dates'].unique()

for date in unique_dates:
    plt.figure(figsize=(10, 6))
    sns.violinplot(x='varieties', y=data.target.name, hue='treatments', data=df[df['dates'] == date])
    plt.title(f'Boxplot for date: {date}')
    plt.show()